In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
dataset_path = '../../big_data/rb-suppliers-challange/receipts.csv'
texts_path = '../../big_data/rb-suppliers-challange/texts/'

In [61]:
dataset = pd.read_csv(dataset_path, index_col=['id'])

dataset = dataset[~dataset.supplier_name.isnull()]
dataset = dataset[~dataset.currency_code.isnull()]
np.random.seed(1)
shuffled = dataset.reindex(np.random.permutation(dataset.index))

unique_supplier_names = shuffled.supplier_name.unique()
unique_currency_codes = shuffled.currency_code.unique()

In [62]:
from os import listdir
from os.path import isfile, join
import json

In [63]:
file_names = [f.split(".")[0] for f in listdir(texts_path) if isfile(join(texts_path, f)) and f.endswith('.json')]
ids = np.array(list(set([int(f) for f in file_names]).intersection(set(dataset.index.values))))
texts = pd.Series(['' for i in ids], index=ids)
for index in ids:
    json_data = open(texts_path + str(index) + '.json', encoding="utf8")
    data = json.load(json_data)
    try:
        texts[index] = data[0]['textAnnotations'][0]['description']
    except:
        texts[index] = ''
dataset['texts'] = texts

In [64]:
import re

In [75]:
prices = pd.Series([[] for i in ids], index=ids)
for index in ids:
    text = dataset['texts'][index]
    total_amount = dataset['total_amount'][index]
    lines = text.split('\n')
    current_prices = []
    for line in lines:
        found_prices = re.findall('\d+[\.,][\d\.,]+', line)
        found_digits = re.findall('\d', line)
        for item in found_prices:
            try:
                current_price = float(item)
            except:
                continue

            new_price = dict()
            new_price['value'] = current_price
            new_price['line'] = line
            new_price['is_total'] = 1 if current_price == total_amount else 0
            new_price['has_total_word'] = 'total' in line.lower()
            new_price['digits_count'] = len(found_digits)

            current_prices.append(new_price)
    
    for i, price in enumerate(current_prices):
        if len(current_prices) == 1:
            price['order'] = 1.00
        else:
            price['order'] = round(i / (len(current_prices) - 1), 2)
        
    prices[index] = current_prices
    
dataset['prices'] = prices

In [66]:
def get_features_array(price):
    features = []
    features.append(price['has_total_word'])
    features.append(price['digits_count'])
    features.append(price['order'])
    return features

In [67]:
X_data = []
Y_data = []

for id_index in ids:
    for price_index, price in enumerate(dataset['prices'][id_index]):
        # new_index = str(id_index) + '_' + str(price_index)
        
        X_data.append(get_features_array(price))
        
        Y_data.append(1 if price['is_total'] else 0)

In [68]:
test_items_count = int(len(X_data) * (10 / 100))
X_train =  np.array(X_data[test_items_count:])
X_test = np.array(X_data[:test_items_count])
Y_train = np.array(Y_data[test_items_count:])
Y_test = np.array(Y_data[:test_items_count])

In [69]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [70]:
total_amount_classifier = RandomForestClassifier()
total_amount_classifier.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [71]:
correct = 0
predictions_for_total = []
predictions_for_non_total = []
for x_t, y_t in zip(X_test, Y_test):
    predicted = total_amount_classifier.predict_proba(x_t.reshape(1, -1))[0][1]
    if y_t == 1:
        predictions_for_total.append(predicted)
    else:
        predictions_for_non_total.append(predicted)
            
print('Average for totals: ', float(sum(predictions_for_total)) / len(predictions_for_total))
print('Average for non totals: ', float(sum(predictions_for_total)) / len(predictions_for_non_total))

Average for totals:  0.36900666586296116
Average for non totals:  0.11842047729699613


In [72]:
def predict_totals(receipt_prices, top_n = 1):
    predicted_totals = []
    for price in receipt_prices:
        features = np.array(get_features_array(price))
        predicted = total_amount_classifier.predict_proba(features.reshape(1, -1))
        predicted_totals.append((price['value'], predicted[0][1]))
                              
    sorted_predictions = sorted(predicted_totals, key=lambda tup: tup[1], reverse = True)
    return list(tup[0] for tup in sorted_predictions[:top_n])

In [74]:
top_n = 1

correct_count = 0
total_count = 0

for index in ids:
    total_amount = dataset['total_amount'][index]
    predictions = predict_totals(dataset['prices'][index], top_n)
    
    total_count += 1
    if total_amount in predictions:
        correct_count += 1
        
print('TOP', str(top_n), 'Accuracy:', round(correct_count / total_count, 2))

TOP 1 Accuracy: 0.47
